# Implementation of the first model: time-weighted feedback model

Let's suppose we have a Matrix C such that each value r_ij is the rating of item j by user i (i.e. if user i clicked / viewed / bought the item j)

Let's suppose we have a Matrix T such that each value t_ij is the time of item j by user i (i.e. the last time that user i did the action r_ij on the item j)

n: number of closest items to a specific item to compute

K: number of clusters

l: number of items to recommend

### Ratings: 

r_ij = 0 iff user i never bought nor viewed nor clicked the item j

r_ij = 1 iff user i clicked the item j

r_ij = 2 iff user i viewed the item j

r_ij = 3 iff user i bought the item j

In [1]:
import numpy as np
import pandas as pd

## Variables

In [2]:
df_ratings = pd.DataFrame(data = {'item0': [0, 2, 1],  'item1': [0, 3, 1], 'item2': [1, 0, 2],  'item3': [2, 3, 0]}, index=['user0', 'user1', 'user2'])
df_ratings

,item0,item1,item2,item3
user0,0,0,1,2
user1,2,3,0,3
user2,1,1,2,0


In [3]:
df_time = pd.DataFrame(data = {'item0': [658, 2350, 15232],  'item1': [735, 3606, 14787], 'item2': [500, 4000, 15986],  'item3': [800, 2006, 14642]}, index=['user0', 'user1', 'user2'])
df_time

,item0,item1,item2,item3
user0,658,735,500,800
user1,2350,3606,4000,2006
user2,15232,14787,15986,14642


In [4]:
C = df_ratings.values
T = df_time.values
n = 2
K = 2
l = 2

In [5]:
C.dtype

dtype('int64')

## Extraction of variables

In [6]:
C = np.load('rating_matrix.npy')
T = np.load('time_matrix.npy')
n = 2
K = 2
l = 2

In [7]:
item_id = pd.read_csv('item_index.csv', header=None)

In [8]:
item_id

,0,1
0,40156,0
1,15632,1
2,36956,2
3,34157,3
4,4953,4
...,...,...
2995,2253,2995
2996,2406,2996
2997,2463,2997
2998,2556,2998


In [9]:
int(item_id[item_id[1] == 4][0])

4953

In [10]:
list(item_id[0])

[40156,
 15632,
 36956,
 34157,
 4953,
 14614,
 32394,
 32769,
 31013,
 14889,
 9186,
 14179,
 4099,
 14784,
 6576,
 43548,
 49272,
 1049,
 7017,
 6445,
 1364,
 2843,
 6909,
 375755,
 6249,
 10858,
 75457,
 332,
 8758,
 11986,
 13831,
 30340,
 32757,
 67334,
 35015,
 36462,
 375069,
 4453,
 6327,
 7101,
 60992,
 9654,
 10274,
 4954,
 8047,
 11542,
 60638,
 131151,
 5285,
 10573,
 14515,
 25769,
 35889,
 67792,
 103581,
 12970,
 30711,
 34952,
 36121,
 36351,
 82984,
 128733,
 129045,
 7771,
 15394,
 30440,
 32138,
 64884,
 3511,
 7258,
 33073,
 35383,
 81684,
 2905,
 6828,
 14520,
 35311,
 36083,
 36780,
 73719,
 76663,
 88855,
 89764,
 381164,
 594,
 768,
 918,
 7107,
 8960,
 14096,
 53294,
 68131,
 79141,
 2866,
 3113,
 7096,
 7492,
 22046,
 34797,
 34972,
 35378,
 35734,
 49920,
 69276,
 70368,
 181750,
 1255,
 1702,
 2322,
 2453,
 5275,
 5400,
 7268,
 7589,
 9582,
 30720,
 35106,
 36992,
 74782,
 90068,
 90884,
 124306,
 126492,
 45,
 488,
 1189,
 1253,
 2633,
 4523,
 6961,
 9460,


In [11]:
df = pd.DataFrame(C, columns = list(item_id[0]))

In [12]:
user_id = pd.read_csv('user_index.csv', header=None)

In [13]:
user_id

,0,1
0,17732,0
1,4094,1
2,20346,2
3,13255,3
4,24034,4
...,...,...
995,57354,995
996,57538,996
997,57676,997
998,70472,998


In [14]:
df.index = list(user_id[0])

In [15]:
[df.columns

SyntaxError: unexpected EOF while parsing (508191303.py, line 1)

## Functions

### Compute item similarity

In [16]:
from scipy import spatial, stats

In [17]:
def cosine_sim2(item1, item2, j):
    '''
    Compute cosine similarity between item1 and item2
    '''
    return 1 - spatial.distance.cosine(item1, item2), int(item_id[item_id[1] == j][0])

In [18]:
def cosine_sim(item1, item2, j):
    '''
    Compute cosine similarity between item1 and item2
    '''
    return 1 - spatial.distance.cosine(item1, item2)

In [19]:
def pearson(item1, item2):
    '''
    Compute pearson correlation coefficient between item1 and item2
    '''
    sim, _ = stats.pearsonr(item1, item2)
    return sim

In [20]:
def prob(item1, item2):
    '''
    Compute conditional probability-based similarity between item1 and item2
    Probability of purchasing item2 given that item1 has been purchased
    '''
    freq1_2 = len([i for i, j in zip(item1, item2) if i == 3 and j == 3])
    freq1 = list(item1).count(3)
    if freq1 == 0:
        return 0
    return freq1_2 / freq1

In [21]:
def compute_item_similarity2(C,n,app='cos'):
    '''
    Compute n closest items for each item in C
    '''
    closest = {}
    for i, item1 in enumerate(C.T):
        print(f'{i}/{len(C.T)}')
        sim = [cosine_sim(item1,item2,j) if i!=j else (-1,j) for j, item2 in enumerate(C.T)]
        #closest[int(item_id[item_id[1] == i][0])] = [t2 for t1, t2 in sorted(sim, key=lambda x: x[0])[:n]]
        #closest[df.columns[i]] = [x for _, x in sorted(zip(sim, range(len(df.columns))), reverse=True)][:n]
        closest[df.columns[i]] = [x for _, x in sorted(zip(sim, df.columns), reverse=True)][:n]
    return closest

In [22]:
def compute_item_similarity(C,n,app='cos'):
    '''
    Compute n closest items for each item in C
    '''
    closest = {}
    for i, item1 in enumerate(C.T):
        print(f'{i}/{len(C.T)}', end='\r')
        sim = [cosine_sim(item1,item2,j) if i!=j else -1 for j, item2 in enumerate(C.T)]
        closest[df.columns[i]] = [df.columns[x] for x in pd.Series(sim).nlargest(n).index.values.tolist()]
    return closest

In [153]:
M = compute_item_similarity(C,n,'cos')
M

{40156: [118410, 30725],
 15632: [725, 10130],
 36956: [76534, 14784],
 34157: [67698, 171245],
 4953: [67334, 33070],
 14614: [79117, 135708],
 32394: [8758, 8759],
 32769: [2866, 6445],
 31013: [9992, 3093],
 14889: [14890, 31981],
 9186: [16652, 4540],
 14179: [8047, 7025],
 4099: [47534, 9739],
 14784: [42775, 1281],
 6576: [36463, 19906],
 43548: [53307, 135708],
 49272: [35458, 4515],
 1049: [3451, 58958],
 7017: [46871, 7015],
 6445: [5352, 32769],
 1364: [31083, 31579],
 2843: [32756, 10951],
 6909: [375205, 66175],
 375755: [130822, 20484],
 6249: [14000, 10354],
 10858: [36602, 34911],
 75457: [70487, 21890],
 332: [34915, 37940],
 8758: [9867, 32394],
 11986: [41310, 2896],
 13831: [6327, 2863],
 30340: [64100, 38133],
 32757: [10957, 10274],
 67334: [4953, 14952],
 35015: [768, 81684],
 36462: [58181, 68986],
 375069: [7270, 34797],
 4453: [34968, 70797],
 6327: [1055, 13831],
 7101: [12445, 63532],
 60992: [146397, 57880],
 9654: [30792, 36642],
 10274: [3148, 5819],
 4954

In [149]:
M

{40156: [2349, 2121],
 15632: [2975, 2013],
 36956: [2276, 13],
 34157: [1122, 2415],
 4953: [33, 1269],
 14614: [2282, 1882],
 32394: [28, 1657],
 32769: [93, 19],
 31013: [608, 140],
 14889: [897, 475],
 9186: [146, 712],
 14179: [44, 2518],
 4099: [2719, 1023],
 14784: [1288, 1168],
 6576: [1088, 756],
 43548: [365, 1882],
 49272: [2675, 1389],
 1049: [1381, 316],
 7017: [274, 286],
 6445: [1192, 7],
 1364: [553, 1262],
 2843: [170, 453],
 6909: [1857, 1799],
 375755: [2882, 1246],
 6249: [893, 211],
 10858: [411, 921],
 75457: [572, 625],
 332: [409, 804],
 8758: [2011, 6],
 11986: [2198, 1625],
 13831: [38, 231],
 30340: [152, 2694],
 32757: [1670, 42],
 67334: [4, 2587],
 35015: [85, 72],
 36462: [943, 249],
 375069: [1402, 98],
 4453: [305, 173],
 6327: [2981, 30],
 7101: [537, 568],
 60992: [2392, 2240],
 9654: [147, 309],
 10274: [233, 717],
 4954: [1291, 1100],
 8047: [11, 542],
 11542: [911, 1321],
 60638: [497, 2813],
 131151: [624, 974],
 5285: [231, 1325],
 10573: [905, 3

### Learning parameters

In [23]:
import math

In [24]:
def f(t, l):
    try:
        return math.exp(-l*t)
    except:
        return -1 

In [25]:
def pred_time(user, item1, i, j, t):
    '''
    Predict opinion of user i on item j
    '''
    
    s1, s2 = 0, 0
    for c in M[df.columns[j]]:
        p = float(C[i][item_id[item_id[0] == c][1]]) # pred(user, item2, i, c, t)
        q = cosine_sim(item1, C.T[item_id[item_id[0] == c][1]], j)*f(T[i][item_id[item_id[0] == c][1]], 1/t)
        s1 += p*q
        s2 += q
    return s1 / s2

In [26]:
item_id[item_id[0] == 118410][1]

2349    2349
Name: 1, dtype: int64

In [38]:
def pred(user, item1, i, j, M):
    '''
    Predict opinion of user i on item j
    '''
    s1, s2 = 0, 0
    for c in M[df.columns[j]]:
        p = float(C[i][item_id[item_id[0] == c][1]]) # pred(user, item2, i, c, t)
        q = cosine_sim(item1, C.T[item_id[item_id[0] == c][1]], j)
        s1 += p*q
        s2 += q
    return s1 / s2

In [28]:
def find_T02(user, i):
    s, m, t_0 = 0, 0, 100
    for j, item in enumerate(C.T):
        pred_ij = pred_time(user, item, i, j, t_0)
        q_ij = C[i][item_id[item_id[0] == j][1]]
        s += abs(pred_ij - q_ij) / len(C.T)
    return t_0

In [29]:
print(np.amax(T) - np.amin(T))
print(np.amin(T))
np.unique(T)[:-1]

255
0


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 122, 123, 124, 126, 127, 130, 131, 134, 135, 138,
       144, 147, 148, 149], dtype=uint8)

In [30]:
T

array([[255, 255, 255, ..., 255,  72, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255,  21, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [31]:
def find_T0(user, i, T):
    s, m, t_0 = 0, 564124554, 1
    for t in range(1, max(np.unique(T)[:-1]) - min(np.unique(T)[:-1])):
        if t == 0:
            continue
        s = 0
        for j, item in enumerate(C.T):
            pred_ij = pred_time(user, item, i, j, t)
            q_ij = C[i][item_id[item_id[0] == j][1]]
            s += abs(pred_ij - q_ij) / len(C.T)
        if s < m:
            m = s
            t_0 = t
    return t_0

In [32]:
def learning_parameters(C,M,T,n,K,l):
    '''
    Search T0 for each user such that the error is minimum
    '''
    parameters = {}
    nb = len(C)
    for i, user in enumerate(C):
        print(f'{i}/{nb}', end='\r')
        parameters[df.index[i]] = find_T0(user, i, T)
    return parameters

In [189]:
learning_parameters(C,M,T,n,K,l)

/tmp/ipykernel_3458/3480545298.py:12: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2
/tmp/ipykernel_3458/3993797755.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if s < m:


KeyboardInterrupt: 

In [115]:
def n_max(l, n):
    maximums = []
    for i in range(n):
        m = max(l)
        maximums.append(m)
        l.remove(m)
    return maximums

In [ ]:
users_df = 

In [43]:
df.index[0]

17732

In [97]:
n = 3

In [98]:
M = compute_item_similarity(C,n,'cos')

In [73]:
def predict_items(C, n, l, M):
    #M = compute_item_similarity(C,n,'cos')
    predictions = {}
    for i, user in enumerate(C):
        print(f'{i}/{len(C)}', end="\r")
        pred_i = []
        for j, item in enumerate(C.T):
            pred_i.append(pred(user, item, i, j, M))
        
        predictions[df.index[i]] = [df.columns[x] for x in pd.Series(pred_i).nlargest(n).index.values.tolist()]
    return predictions

In [74]:
predictions = predict_items(C, n, l, M)

/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


...

/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


/tmp/ipykernel_3079/2671635211.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return s1 / s2


In [75]:
predictions

{17732: [128872, 10588, 67762, 10805],
 4094: [79218, 4391, 87141, 46841],
 20346: [58581, 380081, 379857, 7271],
 13255: [198, 31951, 3131, 301329],
 24034: [58114, 4169, 6817, 50994],
 25480: [5582, 6848, 31981, 23724],
 22489: [18453, 6076, 173413, 166229],
 19341: [133052, 19556, 22218, 7771],
 21156: [51459, 6579, 274526, 31378],
 21634: [375966, 13630, 5115, 6428],
 32944: [136360, 69587, 10349, 76002],
 21049: [2556, 26928, 33602, 116311],
 28242: [8397, 34154, 6313, 35282],
 20148: [52319, 41580, 22615, 84788],
 7112: [14747, 17828, 33695, 7707],
 22852: [9371, 10274, 3148, 10951],
 17987: [69013, 6639, 59931, 7703],
 20353: [35338, 36561, 36772, 3334],
 17842: [79910, 58579, 181749, 5155],
 18950: [196226, 58398, 79995, 35369],
 20942: [36884, 382351, 13941, 58638],
 25666: [179386, 2601, 99541, 377531],
 109: [206092, 211659, 18965, 206093],
 10346: [67846, 35954, 29501, 36319],
 1723: [8328, 4535, 13958, 32606],
 26826: [9037, 12072, 11815, 2176],
 17829: [402831, 13656, 683

In [77]:
import pickle
#with open('predictions4.pkl','wb') as f:
 #   pickle.dump(predictions, f)

In [78]:
import json

In [79]:
with open("predictions4.pkl", 'rb') as f:
    predictions = pickle.load(f)

In [80]:
predictions

{17732: [128872, 10588, 67762, 10805],
 4094: [79218, 4391, 87141, 46841],
 20346: [58581, 380081, 379857, 7271],
 13255: [198, 31951, 3131, 301329],
 24034: [58114, 4169, 6817, 50994],
 25480: [5582, 6848, 31981, 23724],
 22489: [18453, 6076, 173413, 166229],
 19341: [133052, 19556, 22218, 7771],
 21156: [51459, 6579, 274526, 31378],
 21634: [375966, 13630, 5115, 6428],
 32944: [136360, 69587, 10349, 76002],
 21049: [2556, 26928, 33602, 116311],
 28242: [8397, 34154, 6313, 35282],
 20148: [52319, 41580, 22615, 84788],
 7112: [14747, 17828, 33695, 7707],
 22852: [9371, 10274, 3148, 10951],
 17987: [69013, 6639, 59931, 7703],
 20353: [35338, 36561, 36772, 3334],
 17842: [79910, 58579, 181749, 5155],
 18950: [196226, 58398, 79995, 35369],
 20942: [36884, 382351, 13941, 58638],
 25666: [179386, 2601, 99541, 377531],
 109: [206092, 211659, 18965, 206093],
 10346: [67846, 35954, 29501, 36319],
 1723: [8328, 4535, 13958, 32606],
 26826: [9037, 12072, 11815, 2176],
 17829: [402831, 13656, 683

In [81]:
rating = np.load("Intelligent-systems-and-recommendations/rating_matrix.npy")
itemID = pd.read_csv("Intelligent-systems-and-recommendations/item_index.csv", header=None)
userID = pd.read_csv("Intelligent-systems-and-recommendations/user_index.csv", header=None)

In [82]:
df = pd.DataFrame(rating, columns = list(itemID[0]))
df.index = list(userID[0])

In [83]:
items = list(itemID[0])
items_df = []
users_df = []

In [84]:
for user in df.index:        # loop over rows of df (users)
    for index, rating in enumerate(list(df.loc[user])):   # loop over columns (items) and remember index
        if rating == 3:         # compare rating
            item = items[index]    # get itemID in list of items, at the index observed
            items_df.append(item)  # append the list of items
            users_df.append(user)  # append the list of users
            
gt = pd.DataFrame(list(zip(users_df, items_df, [1]*len(items_df))), columns =["User", "Item", "Relevance"])

In [85]:
users = []
items = []
for user in predictions.keys():
    users.extend([user]*len(predictions[user]))
    items.extend(predictions[user])

In [91]:
df_results = pd.DataFrame(list(zip(users, items, [1, 2, 3, 4]*len(users))), columns =["User", "Item","Rank"])

In [92]:
gt

,User,Item,Relevance
0,22489,25189,1
1,22489,92881,1
2,22489,25307,1
3,21156,80781,1
4,7112,9313,1
...,...,...,...
79,78460,103581,1
80,6741,11664,1
81,54700,76977,1
82,24205,84752,1


In [93]:
df_results

,User,Item,Rank
0,17732,128872,1
1,17732,10588,2
2,17732,67762,3
3,17732,10805,4
4,4094,79218,1
...,...,...,...
3995,70472,10467,4
3996,71775,14072,1
3997,71775,38354,2
3998,71775,12,3


In [94]:
labeled_search_results = df_results.merge(gt, how='left', on=['User', 'Item']).fillna(0)

In [95]:
labeled_search_results

,User,Item,Rank,Relevance
0,17732,128872,1,0.0
1,17732,10588,2,0.0
2,17732,67762,3,0.0
3,17732,10805,4,0.0
4,4094,79218,1,0.0
...,...,...,...,...
3995,70472,10467,4,0.0
3996,71775,14072,1,0.0
3997,71775,38354,2,0.0
3998,71775,12,3,0.0


In [96]:
relevances_rank = labeled_search_results.groupby(['User', 'Relevance'])['Rank'].min()

ranks = relevances_rank.loc[:, 1]

reciprocal_ranks = 1 / (ranks)

reciprocal_ranks.mean()

0.5895061728395061